In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
from sklearn.ensemble import BaggingRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import os
from pprint import pprint


In [164]:
tr = pd.read_csv("/Users/yassamandavilu/Downloads/train1.csv", index_col = 0)
ts = pd.read_csv("/Users/yassamandavilu/Downloads/test1.csv", index_col = 0)
newtrain = train[['Lot Area','Overall Qual','Overall Cond','Total Bsmt SF','1st Flr SF','2nd Flr SF','Gr Liv Area','Garage Area','Wood Deck SF','SalePrice','Overall Qual','Year Built','Bedroom AbvGr','TotRms AbvGrd','Fireplaces','Garage Cars']]
newtrain2 = train[['1st Flr SF','2nd Flr SF','SalePrice','Lot Area','Garage Area','Overall Qual','Year Built','Bedroom AbvGr','TotRms AbvGrd','Fireplaces','Garage Cars']]

def process(train):
    train = train.dropna() 
    mu = np.mean(train['1st Flr SF'])
    std = np.std(train['1st Flr SF'])
    normalized_data = (train['1st Flr SF'] - mu)/std
    indexes = normalized_data < 3
    train_cleaned = train.loc[indexes, :]
    return train_cleaned

In [151]:
train_null = tr.isna().sum()
print('Columns that contain missing values in training dataset')
train_null[train_null > 0]

test_null = ts.isna().sum()
print('Columns that contain missing values in test dataset')
test_null[test_null > 0]

Columns that contain missing values in training dataset
Columns that contain missing values in test dataset


Lot Frontage      158
Alley             860
Mas Vnr Type        7
Mas Vnr Area        7
Bsmt Qual          27
Bsmt Cond          27
Bsmt Exposure      27
BsmtFin Type 1     27
BsmtFin Type 2     27
Bsmt Full Bath      1
Bsmt Half Bath      1
Fireplace Qu      486
Garage Type        53
Garage Yr Blt      53
Garage Finish      53
Garage Qual        53
Garage Cond        53
Pool QC           928
Fence             725
Misc Feature      895
dtype: int64

In [152]:
tt = [tr, ts]

# Filling in missing values for columns that contain missing values in both
# training and test datasets
for df in tt:
    df['Lot Frontage'].fillna(df['Lot Frontage'].mean(), inplace=True)
    df['Alley'].fillna('No alley', inplace=True)
    df['Mas Vnr Area'].fillna(df['Mas Vnr Area'].median(), inplace=True)
    df['Fireplace Qu'].fillna('No fireplace', inplace=True)
    df['Pool QC'].fillna('No pool', inplace=True)
    df['Fence'].fillna('No fence', inplace=True)
    df['Misc Feature'].fillna('No', inplace=True)
    df['Garage Yr Blt'].fillna(1, inplace=True)
    for a in ['Mas Vnr Type', 'Bsmt Exposure']:
        df[a].fillna(df[a].mode().iloc[0], inplace=True)
    for a in ['Bsmt Qual', 'Bsmt Cond', 'BsmtFin Type 1', 'BsmtFin Type 2']:
        df[a].fillna('No basement', inplace=True)
    for a in ['Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond']:
        df[a].fillna('No Garage', inplace=True)

In [153]:
train_null = tr.isna().sum()
print('Columns that contain missing values in training dataset')
train_null[train_null > 0]

test_null = ts.isna().sum()
print('Columns that contain missing values in test dataset')
test_null[test_null > 0]

Columns that contain missing values in training dataset
Columns that contain missing values in test dataset


Bsmt Full Bath    1
Bsmt Half Bath    1
dtype: int64

In [154]:
for df in tt:
    df['MS SubClass'] = df['MS SubClass'].astype(str)
    mp1 = {'Ex':4,'Gd':3,'TA':2,'Fa':1,'Po':0}
    df['Exter Qual'] = df['Exter Qual'].map(mp1)
    df['Exter Cond'] = df['Exter Cond'].map(mp1)
    mp3 = {'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No basement':0}
    df['Bsmt Qual'] = df['Bsmt Qual'].map(mp3)
    df['Bsmt Cond'] = df['Bsmt Cond'].map(mp3)
    df['Bsmt Exposure'] = df['Bsmt Exposure'].map({'Gd':4,'Av':3,'Mn':2,'No':1,'No basement':0})
    mp2 = {'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'No basement':0}
    df['BsmtFin Type 1'] = df['BsmtFin Type 1'].map(mp2)
    df['BsmtFin Type 2'] = df['BsmtFin Type 2'].map(mp2)
    df['Heating QC'] = df['Heating QC'].map(mp1)
    df['Central Air'] = df['Central Air'].map({'Y':1,'N':0})
    df['Kitchen Qual'] = df['Kitchen Qual'].map(mp1)
    df['Functional'] = df['Functional'].map({'Typ':7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal':0})
    df['Garage Finish'] = df['Garage Finish'].map({'Fin':3,'RFn':2,'Unf':1,'No Garage':0})
    df['Garage Qual'] = df['Garage Qual'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
    df['Garage Cond'] = df['Garage Cond'].map({'Ex':5,'Gd':4,'TA':3,'Fa':2,'Po':1,'No Garage':0})
    df['Pool QC'] = df['Pool QC'].map({'Ex':4,'Gd':3,'TA':2,'Fa':1,'No pool':0})
    df['Fence'] = df['Fence'].map({'GdPrv':4,'MnPrv':3,'GdWo':2,'MnWw':1,'No fence':0})


In [181]:
train2 = tt[0]
test2 = tt[1]
train2.isna().sum()

MS SubClass       0
MS Zoning         0
Lot Frontage      0
Lot Area          0
Street            0
Alley             0
Lot Shape         0
Land Contour      0
Utilities         0
Lot Config        0
Land Slope        0
Neighborhood      0
Condition 1       0
Condition 2       0
Bldg Type         0
House Style       0
Overall Qual      0
Overall Cond      0
Year Built        0
Year Remod/Add    0
Roof Style        0
Roof Matl         0
Exterior 1st      0
Exterior 2nd      0
Mas Vnr Type      0
Mas Vnr Area      0
Exter Qual        0
Exter Cond        0
Foundation        0
Bsmt Qual         0
                 ..
Bedroom AbvGr     0
Kitchen AbvGr     0
Kitchen Qual      0
TotRms AbvGrd     0
Functional        0
Fireplaces        0
Fireplace Qu      0
Garage Type       0
Garage Yr Blt     0
Garage Finish     0
Garage Cars       1
Garage Area       1
Garage Qual       0
Garage Cond       0
Paved Drive       0
Wood Deck SF      0
Open Porch SF     0
Enclosed Porch    0
3Ssn Porch        0


In [155]:
tr['Electrical'].fillna(tr['Electrical'].mode().iloc[0], inplace=True)

for a in ['Utilities', 'MS Zoning', 'Exterior 1st', 'Exterior 2nd', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Kitchen Qual',
         'Functional', 'Garage Cars', 'Sale Type']:
    ts[a].fillna(ts[a].mode().iloc[0], inplace=True)

for a in ['BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Garage Area']:
    ts[a].fillna(ts[a].median(), inplace=True)

In [1]:
new = process(newtrain2)
X = new.drop('SalePrice', axis = 1)
y = new['SalePrice']

s = sm.add_constant(X)
pred = sm.OLS(y,s).fit()
p = pred.params


NameError: name 'process' is not defined

In [159]:
test = pd.read_csv("/Users/yassamandavilu/Downloads/test1.csv", index_col = 0)
newtest = test[['Lot Area','Overall Qual','Overall Cond','Total Bsmt SF','1st Flr SF','2nd Flr SF','Gr Liv Area','Garage Area','Wood Deck SF','Bedroom AbvGr','TotRms AbvGrd','Fireplaces','Garage Cars','Central Air']]
newtest2 = test[['1st Flr SF','2nd Flr SF','Lot Area','Garage Area','Overall Qual','Year Built','Bedroom AbvGr','TotRms AbvGrd','Fireplaces','Garage Cars','Central Air']]

In [160]:
def linreg(x,y,x2):
    s = sm.add_constant(x)
    r = sm.OLS(y,s).fit()
    x2 = sm.add_constant(x2)
    ypred = r.predict(x2)
    return ypred

In [161]:
test_pred = linreg(X,y,newtest2)
test['SalePrice'] = test_pred
#test.to_csv(r"C:\Users\Duker\Desktop\CS530\Quiz\Quiz 2\attempt3.csv")

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [162]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1 - r_squared)
    return vif

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    vif_scores = []
    var = []
    
    new_data = processed_train.drop(['SalePrice'], axis = 1)
    
    for i in new_data.columns:
        
        var = [i]
        x = new_data.drop(i,axis=1).values
        y = new_data[i].values
        
        #calculate regression
        s = sm.add_constant(x)
        results = sm.OLS(y,s).fit()
        r2 = results.rsquared
        
        vif = round(calculate_vif(r2),3)
        var.append(vif)
        vif_scores.append(var)
        
        vif_dataframe = pd.DataFrame(vif_scores, columns = ['Variables', 'VIF'])   
    
    return vif_dataframe

In [163]:
vif = generate_vif_dataframe(new)
vif

TypeError: unsupported operand type(s) for -: 'str' and 'str'